<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/smallmolecules-Pradeep/bioco_smallmolecules_semiveiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna
!pip install LAMDA-SSL

Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts_semi

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av res

In [ ]:
# Oppretter instans for lagring av res
res_semi = Resultat_dicts_semi()

# Setter fast treningsandel og faste markerte andeler

In [ ]:
# Fast treningsandel
trenings_andel = 1.0

# Markerte andeler til undersøkelse
andel_av_markert = [0.5, 1.0, 10.0]

# Parametervalg for hyperoptimalisering

In [ ]:
# Antall runder for hyperoptimalisering
n_runder_hel = 100
n_runder_andel = 100

# Antall gjentagelser
n_gjentagelser = 2

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 20% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av satt markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andel
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                          dict_res=res_semi.dict_res_1_hel,
                          rå_data=rå_data,
                          treningssett_markert=trening_markert,
                          enzymtyper_treningssett_markert=enzymtyper_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          n_splitt=n_splitt,
                          n_gjentagelser=n_gjentagelser,
                          n_runder=n_runder_andel,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

  # Lagring av resulatater
  res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
  res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

  res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_semi.csv')
  res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_opt_semi.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                            dict_res=res_semi.dict_res_2_hel,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

  # Lagring av resultater
  res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
  res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

  res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_semi.csv')
  res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_opt_semi.csv')

## Designproduksjon

## Alternativ 1: 80% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)


  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                            dict_res=res_semi.dict_res_1_design,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            enzymtyper_treningssett_markert=enzymtyper_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_gjentagelser=n_gjentagelser,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

  # Lagring av resulatater
  res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
  res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

  res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_semi.csv')
  res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_opt_semi.csv')

[I 2024-05-03 00:58:53,541] A new study created in memory with name: no-name-5a8d4abd-f437-4507-8cba-63005add9bf2
[I 2024-05-03 00:59:34,312] Trial 0 finished with value: 0.9432494812590041 and parameters: {'k1': 9, 'k2': 4, 'p1': 9, 'p2': 9, 'max_iters': 2, 'pool_size': 100}. Best is trial 0 with value: 0.9432494812590041.
[I 2024-05-03 01:00:31,677] Trial 1 finished with value: 1.2974444166218213 and parameters: {'k1': 20, 'k2': 20, 'p1': 1, 'p2': 2, 'max_iters': 27, 'pool_size': 14}. Best is trial 0 with value: 0.9432494812590041.
[I 2024-05-03 01:01:01,947] Trial 2 finished with value: 0.8191104689416712 and parameters: {'k1': 1, 'k2': 6, 'p1': 9, 'p2': 3, 'max_iters': 10, 'pool_size': 33}. Best is trial 2 with value: 0.8191104689416712.
[I 2024-05-03 01:02:29,442] Trial 3 finished with value: 0.9434079666457058 and parameters: {'k1': 16, 'k2': 2, 'p1': 3, 'p2': 9, 'max_iters': 42, 'pool_size': 25}. Best is trial 2 with value: 0.8191104689416712.
[I 2024-05-03 01:04:29,080] Trial 5

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
B                 39  20.97
E                 39  20.97
C                 38  20.43
D                 37  19.89
A2                29  15.59
A1                 4   2.15
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 36  24.66
C                 28  19.18
B                 26  17.81
D                 23  15.75
A2                19  13.01
A1                14   9.59
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
D               1926  19.01
A1              1826  18.02
B               1728  17.05
C               1699  16.77
E               1631  16.09
A2              1324  13.06


In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  #Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                        dict_res=res_semi.dict_res_2_design,
                        rå_data=rå_data,
                        treningssett_markert=trening_markert,
                        treningssett_umarkert=treningssett_umarkert,
                        treningssett=treningssett,
                        testsett=testsett,
                        n_splitt=n_splitt,
                        n_runder=n_runder_andel,
                        test_andel=test_andel,
                        trenings_andel=trenings_andel,
                        random_seed=random_seed)

  # Lagring av resultater
  res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
  res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

  res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_semi.csv')
  res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_opt_semi.csv')

[I 2024-05-03 13:09:03,037] A new study created in memory with name: no-name-6f948776-5b07-4e51-8467-89ddf5ae5f55
[I 2024-05-03 13:13:53,895] Trial 1 finished with value: 0.7332144691259612 and parameters: {'k1': 18, 'k2': 4, 'p1': 1, 'p2': 4, 'max_iters': 20, 'pool_size': 170}. Best is trial 1 with value: 0.7332144691259612.
[I 2024-05-03 13:18:07,547] Trial 0 finished with value: 0.8464390799462637 and parameters: {'k1': 3, 'k2': 15, 'p1': 5, 'p2': 9, 'max_iters': 36, 'pool_size': 55}. Best is trial 1 with value: 0.7332144691259612.
[I 2024-05-03 13:20:11,211] Trial 2 finished with value: 0.7199390420204909 and parameters: {'k1': 16, 'k2': 3, 'p1': 2, 'p2': 5, 'max_iters': 40, 'pool_size': 91}. Best is trial 2 with value: 0.7199390420204909.
[I 2024-05-03 13:21:04,210] Trial 3 finished with value: 0.7921181746070953 and parameters: {'k1': 15, 'k2': 4, 'p1': 2, 'p2': 9, 'max_iters': 12, 'pool_size': 70}. Best is trial 2 with value: 0.7199390420204909.
[I 2024-05-03 13:23:32,855] Trial

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

### Fast testandel

In [ ]:
#test_andel
test_andel = 0.2

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt_NIR,
                         dict_res=res_semi.dict_res_1_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_NIR)
res_1_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_opt_NIR)

res_1_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_semi_NIR.csv')
res_1_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Manuell fordeling

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt_NIR,
                         dict_res=res_semi.dict_res_2_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

res_2_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_NIR)
res_2_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_opt_NIR)

res_2_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_semi_NIR.csv')
res_2_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_opt_semi_NIR.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt_NIR,
                         dict_res=res_semi.dict_res_1_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_design_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_NIR)
res_1_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_opt_NIR)

res_1_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_semi_NIR.csv')
res_1_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt_NIR,
                         dict_res=res_semi.dict_res_2_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_design_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_NIR)
res_2_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_opt_NIR)

res_2_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_semi_NIR.csv')
res_2_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_opt_semi_NIR.csv')

Beste modell